In [6]:
# %% [markdown]
# # Generate Full Swedish Stock Company CSV (with Retry)
#
# Produces `sweden_company_data.csv` and `errors_log.csv`.
# 1. Uses **investpy** for *all* Swedish tickers
# 2. Resumes from any existing CSV
# 3. Retries any tickers in the prior `errors_log.csv`
# 4. Sleeps between calls to avoid rate-limit overload
# ----

# %%
# 1) Imports and globals
import investpy
import pandas as pd
import yfinance as yf
import time
import os

OUTPUT_CSV = "sweden_company_data.csv"
ERROR_LOG_CSV = "errors_log.csv"
SLEEP_SEC = 1.0   # adjust if you hit rate-limits

# %% [markdown]
# ## 2) Fetch **all** Swedish tickers via investpy

# %%

stocks_df = investpy.get_stocks(country="Sweden")
stocks_df['YahooTicker'] = stocks_df['symbol'].str.upper() + ".ST"
stocks_df['Tickersymbol'] = stocks_df['YahooTicker']
stocks_df['CompanyName'] = stocks_df['name']

tickers_list = stocks_df['YahooTicker'].tolist()
print(f"Found {len(tickers_list)} Swedish tickers via investpy.")

# %% [markdown]
# ## 3) Load or initialize checkpoint CSV & previous errors

# %%
# — load successes
if os.path.exists(OUTPUT_CSV):
    df_out = pd.read_csv(OUTPUT_CSV)
    done = set(df_out['Tickersymbol'].astype(str))
    print(f"Resuming – {len(done)} tickers already done.")
else:
    df_out = pd.DataFrame(columns=[
        "Tickersymbol", "YahooTicker", "CompanyName", "Exchange"
    ])
    done = set()
    print("Starting fresh – no existing CSV found.")

# — load previous errors to retry
if os.path.exists(ERROR_LOG_CSV):
    prev_err = pd.read_csv(ERROR_LOG_CSV)
    retry_tickers = prev_err['Tickersymbol'].astype(str).tolist()
    print(f"Will retry {len(retry_tickers)} previously failed tickers.")
else:
    retry_tickers = []

# %% [markdown]
# ## 4) Build ordered list: retry failures first, then new tickers

# %%
to_retry = [t for t in retry_tickers if t not in done]
to_new = [t for t in tickers_list if t not in done and t not in retry_tickers]
to_process = to_retry + to_new

print(f"{len(to_retry)} to retry, {len(to_new)} new → total {len(to_process)} tickers to process.")

# %%
# %% [markdown]
# ## 5) Loop over `to_process`, fetch info, checkpoint, and collect fresh errors
#
# Now with a fallback for yfinance returning None.

# %%
errors = []

for ticker in to_process:
    print(f"→ Processing {ticker} …", end="", flush=True)
    try:
        info = yf.Ticker(ticker).info

        # if info is None, force into our AttributeError handler below
        if info is None:
            raise AttributeError("info is None")

        # normal path
        name = info.get("shortName") or info.get("longName") or ticker
        exchange = info.get("exchange", "Unknown")

        new_row = pd.DataFrame([{
            "Tickersymbol": ticker,
            "YahooTicker":  ticker,
            "CompanyName":  name,
            "Exchange":     exchange
        }])
        df_out = pd.concat([df_out, new_row], ignore_index=True)

        # checkpoint successes
        df_out.to_csv(OUTPUT_CSV, index=False)
        done.add(ticker)
        print(" done")

    except AttributeError as e:
        msg = str(e)
        if "NoneType" in msg or "info is None" in msg:
            # fallback: record with investpy name, blank exchange
            fallback_name = stocks_df.loc[
                stocks_df['YahooTicker'] == ticker, 'CompanyName'
            ].iat[0]
            new_row = pd.DataFrame([{
                "Tickersymbol": ticker,
                "YahooTicker":  ticker,
                "CompanyName":  fallback_name,
                "Exchange":     ""
            }])
            df_out = pd.concat([df_out, new_row], ignore_index=True)
            df_out.to_csv(OUTPUT_CSV, index=False)
            done.add(ticker)
            print(" fallback → recorded without yfinance info")
        else:
            # truly unexpected AttributeError
            print(f" ERROR: {msg}")
            errors.append({"Tickersymbol": ticker, "Error": msg})

    except Exception as e:
        # all other exceptions still get logged for retry
        msg = str(e)
        print(f" ERROR: {msg}")
        errors.append({"Tickersymbol": ticker, "Error": msg})

    finally:
        time.sleep(SLEEP_SEC)

print("Data collection pass complete.")


# %% [markdown]
# ## 6) Save updated error log

# %%
if errors:
    pd.DataFrame(errors).to_csv(ERROR_LOG_CSV, index=False)
    print(f"Logged {len(errors)} errors to {ERROR_LOG_CSV}")
else:
    # remove old log if everything succeeded
    if os.path.exists(ERROR_LOG_CSV):
        os.remove(ERROR_LOG_CSV)
    print("No errors on this run.  👍")

Found 1004 Swedish tickers via investpy.
Resuming – 1004 tickers already done.
0 to retry, 0 new → total 0 tickers to process.
Data collection pass complete.
No errors on this run.  👍
